#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import users table:

In [2]:
users = pd.read_csv('users_table.csv')

#### 3. Rename Id column to userId

In [3]:
users.rename(columns={'Id': 'userId'}, inplace= True)

#### 4. Import posts table:

In [4]:
posts = pd.read_csv('posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [5]:
posts.rename(columns= {'Id': 'postId', 'OwnerUserId': 'userId'}, inplace= True)

#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [6]:
users = users[['userId', 'Reputation', 'Views', 'UpVotes', 'DownVotes']]
posts = posts[['postId', 'Score', 'userId', 'ViewCount', 'CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [7]:
users_posts = users.merge(posts, on='userId')

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [9]:
print(users_posts.isnull().sum())
# 23572 missing values on the ViewCount column

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64


#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [11]:
#it is unlikely that there are no posts with zero views I think it is safe to assume NaNs are zeros, so I will fill them with 0
users_posts['ViewCount'].fillna(0, inplace=True)
print(users_posts['ViewCount'].value_counts())

0.0       23572
150.0        43
98.0         43
122.0        42
156.0        41
          ...  
4414.0        1
9772.0        1
1796.0        1
4976.0        1
2174.0        1
Name: ViewCount, Length: 3403, dtype: int64


#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [12]:
users_posts = users_posts.astype({'ViewCount': 'int64'})
users_posts.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object